<a href="https://colab.research.google.com/github/maximecharriere/CryptoWallet/blob/main/CryptoWallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import csv
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json